In [1]:
from surprise import SVD, Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate
from collections import defaultdict
import pandas as pd

In [10]:
df = pd.read_csv("../ALS/data/ratings.csv", nrows=1000000)  # Header wird automatisch erkannt
df.columns = ["user", "item", "rating"]  # Umbenennen, falls nötig

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[["user", "item", "rating"]], reader)  # Spaltennamen statt Indizes

In [11]:
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8713  0.8703  0.8715  0.8677  0.8687  0.8699  0.0015  
MAE (testset)     0.6815  0.6817  0.6812  0.6795  0.6795  0.6807  0.0010  
Fit time          3.70    4.34    3.92    4.47    4.13    4.11    0.28    
Test time         0.62    0.62    0.61    0.71    0.65    0.64    0.04    


{'test_rmse': array([0.87132088, 0.8703213 , 0.87145765, 0.86765762, 0.86874061]),
 'test_mae': array([0.68154045, 0.681741  , 0.68119256, 0.67949982, 0.67954632]),
 'fit_time': (3.701836109161377,
  4.341921091079712,
  3.924405097961426,
  4.467318773269653,
  4.125885963439941),
 'test_time': (0.6189332008361816,
  0.6157870292663574,
  0.6067540645599365,
  0.7122130393981934,
  0.6460320949554443)}

In [ ]:
algo = KNNBasic()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9093  0.9097  0.9120  0.9103  0.0012  
MAE (testset)     0.7170  0.7172  0.7185  0.7176  0.0006  
Fit time          8.05    8.25    8.02    8.11    0.10    
Test time         64.86   66.65   62.73   64.75   1.60    


{'test_rmse': array([0.90926767, 0.90971025, 0.91200533]),
 'test_mae': array([0.71704404, 0.7171998 , 0.71848078]),
 'fit_time': (8.053538084030151, 8.247802019119263, 8.016561031341553),
 'test_time': (64.85671186447144, 66.65344619750977, 62.732341051101685)}

In [11]:

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
# Train an SVD using test and split dataset
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

In [ ]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])